In [ ]:
'''Train a model for the CY-metric'''

In [1]:
# Generate training data
import numpy as np
import os as os
import pickle as pickle
from cymetric.pointgen.pointgen import PointGenerator

monomials = 5*np.eye(5, dtype=np.int64)
coefficients = np.ones(5)
kmoduli = np.ones(1)
ambient = np.array([4])

pg = PointGenerator(monomials, coefficients, kmoduli, ambient)
points = pg.generate_points(100)
#pg.cy_condition(points)

dirname = 'cy_models/train_data'
n_p = 100000
kappa = pg.prepare_dataset(n_p, dirname)
data = np.load(os.path.join(dirname, 'dataset.npz'))
pg.prepare_basis(dirname, kappa=kappa)


/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
pointgen:INFO:Vol_k: 5.000000000000001, Vol_cy: 284.2561104432773.


0

In [2]:
import tensorflow as tf
tfk = tf.keras
from cymetric.models.callbacks import SigmaCallback
from cymetric.models.tfmodels import PhiFSModel
from cymetric.models.metrics import SigmaLoss
from cymetric.models.tfhelper import prepare_tf_basis, train_model

data = np.load(os.path.join(dirname, 'dataset.npz'))
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)
BASIS = prepare_tf_basis(BASIS)

n_out = 1
nlayer = 3
nHidden = 64
act = 'gelu'
nEpochs = 1
bSizes = [64, 50000]
alpha = [1., 1., 1., 1., 1.]
nfold = 3
n_in = 2*5

nn_phi = tf.keras.Sequential()
nn_phi.add(tfk.Input(shape=(n_in)))
for i in range(nlayer):
    nn_phi.add(tfk.layers.Dense(nHidden, activation=act))
nn_phi.add(tfk.layers.Dense(n_out, use_bias=False))
from cymetric.models.tfmodels import PhiFSModel
phimodel = PhiFSModel(nn_phi, BASIS, alpha=alpha)
opt_phi = tfk.optimizers.Adam()
scb = SigmaCallback((data['X_val'], data['y_val']))
cb_list = [scb]
cmetrics = [SigmaLoss()]

phimodel, training_history = train_model(phimodel, data, optimizer=opt_phi, epochs=nEpochs, batch_sizes=bSizes, 
                                       verbose=1, custom_metrics=cmetrics, callbacks=cb_list)


absl:WARNING:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.



Epoch  1/1
1407/1407 [==============================] - 11s 5ms/step - sigma_loss: 0.2201
 - Sigma measure val:      0.1556==] - ETA: 0s - sigma_loss: 0.1527
2/2 [==============================] - 6s 2s/step - sigma_loss: 0.1527 - sigma_val: 0.1556


In [12]:
#Input testing
print(data.keys())
data_sample = data['X_train'][:2]
print(data_sample.shape, phimodel(data_sample).shape)
print(data_sample)
print(phimodel(data_sample))

KeysView(NpzFile 'cy_models/train_data/dataset.npz' with keys: X_train, y_train, X_val, y_val, val_pullbacks)


tensorflow:WARNING:6 out of the last 6 calls to <function pfor.<locals>.f at 0x1602ee4c0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


(2, 10) (2, 3, 3)
[[ 8.30523344e-01 -9.27616916e-02  3.53066910e-02  4.72769731e-01
   1.00000000e+00  5.34721915e-01  5.88295918e-01 -7.30581158e-01
   4.88821359e-01  2.92918737e-18]
 [ 6.62693137e-01  4.60359176e-01  1.00000000e+00  7.73019000e-01
   5.39675140e-01  5.72620595e-01  5.20721633e-01 -5.64361462e-18
   4.36057483e-01  1.59782110e-01]]
tf.Tensor(
[[[ 0.09294742+9.3132257e-10j  0.00564987+4.4295304e-03j
   -0.00334913+5.3939410e-04j]
  [ 0.00564986-4.4295313e-03j  0.0968868 +1.3969839e-09j
   -0.00366766-2.6401198e-03j]
  [-0.00334913-5.3939410e-04j -0.00366765+2.6401216e-03j
    0.09535365+9.3132257e-10j]]

 [[ 0.16272612+2.7939677e-09j  0.02228601-1.3242747e-02j
   -0.00215267+1.7732924e-02j]
  [ 0.02228601+1.3242751e-02j  0.09700379-4.6566129e-10j
   -0.0049596 +6.4791348e-03j]
  [-0.00215267-1.7732929e-02j -0.0049596 -6.4791329e-03j
    0.08899395+9.3132257e-10j]]], shape=(2, 3, 3), dtype=complex64)


In [ ]:
#Save the trained model
save_filepath = os.getcwd()+'/models/cy_models/test_model.keras'
phimodel.save(save_filepath)


In [ ]:
#Import the trained model
imported_model = PhiFSModel(nn_phi, BASIS, alpha=alpha)
imported_model.nn_phi = tfk.models.load_model(save_filepath)
